## Merge and Standarizing Scraped Data

In [10]:
def read_datasets(file_paths):
    """
    Read all datasets and return a dictionary of dataframes
    Now using np.nan instead of "NULL" string
    """
    dfs = {}
    for bank, path in file_paths.items():
        df = pd.read_csv(path)
        print(f"\nColumns in {bank}:")
        print(df.columns.tolist())
        df['bank_source'] = bank
        df['donor_id'] = f"{bank}_" + df.index.astype(str)
        # Using np.nan instead of "NULL" string
        df = df.replace({"NULL": np.nan, "": np.nan})
        dfs[bank] = df
    return dfs

def merge_description_fields(row, dataset):
    """
    Merge different description fields based on dataset structure, separated by periods
    Now handling np.nan values
    """
    descriptions = []
    
    if dataset == 'bank1':
        if pd.notna(row.get('headline')):
            descriptions.append(clean_ids_from_text(row['headline']))
        if pd.notna(row.get('donor_description')):
            descriptions.append(clean_ids_from_text(row['donor_description']))
        if pd.notna(row.get('donor_lookalikes')):
            descriptions.append(clean_ids_from_text(row['donor_lookalikes']))
            
    elif dataset == 'bank2':
        if pd.notna(row.get('donor_description')):
            descriptions.append(clean_ids_from_text(row['donor_description']))
        if pd.notna(row.get('donor_quote')):
            descriptions.append(clean_ids_from_text(row['donor_quote']))
            
    elif dataset == 'bank3':
        if pd.notna(row.get('donor_alias')):
            descriptions.append(clean_ids_from_text(row['donor_alias']))
        if pd.notna(row.get('staff_impression')):
            descriptions.append(clean_ids_from_text(row['staff_impression']))
            
    elif dataset == 'bank4':
        if pd.notna(row.get('description')):
            descriptions.append(clean_ids_from_text(row['description']))
        if pd.notna(row.get('personal goals')):
            descriptions.append(clean_ids_from_text(row['personal goals']))
    
    return '. '.join(filter(None, descriptions))

def clean_and_standardize_df(df, dataset):
    """
    Clean and standardize a single dataframe
    Now using np.nan for missing values
    """
    # Create standardized dataframe with required columns
    standardized = pd.DataFrame()
    
    # Keep donor_id and bank_source for validation
    standardized['donor_id'] = df['donor_id']
    standardized['bank_source'] = df['bank_source']
    
    # Merge description fields
    standardized['donor_description'] = df.apply(lambda x: merge_description_fields(x, dataset), axis=1)
    
    # Map physical attributes
    standardized['height'] = df['height']
    standardized['weight'] = df['weight']
    
    # Handle eye color
    if 'eye_color' in df.columns:
        standardized['eye_color'] = df['eye_color']
    elif 'eye color' in df.columns:
        standardized['eye_color'] = df['eye color']
    else:
        standardized['eye_color'] = np.nan
    
    # Handle hair color
    if 'hair_color' in df.columns:
        standardized['hair_color'] = df['hair_color']
    elif 'hair' in df.columns:
        standardized['hair_color'] = df['hair']
    else:
        standardized['hair_color'] = np.nan
    
    # Handle skin tone
    if 'skin_tone' in df.columns:
        standardized['skin_tone'] = df['skin_tone']
    elif 'skin tone' in df.columns:
        standardized['skin_tone'] = df['skin tone']
    else:
        standardized['skin_tone'] = np.nan
    
    # Handle education
    if 'education_level' in df.columns:
        standardized['education_level'] = df['education_level']
        if 'areas_of_study' in df.columns:
            standardized['education_field'] = df['areas_of_study']
        else:
            standardized['education_field'] = np.nan
    elif 'education' in df.columns:
        standardized['education_level'] = df['education']
        standardized['education_field'] = np.nan
    elif 'degree' in df.columns:
        standardized['education_level'] = df['degree']
        if 'favorite_subject' in df.columns:
            standardized['education_field'] = df['favorite_subject']
        else:
            standardized['education_field'] = np.nan
    else:
        standardized['education_level'] = np.nan
        standardized['education_field'] = np.nan
    
    # Handle ethnic background
    ethnic_fields = []
    if 'ethnic_origin' in df.columns:
        ethnic_fields.append(df['ethnic_origin'])
    if 'mother_ethnic_origin' in df.columns and 'father_ethnic_origin' in df.columns:
        mother_ethnic = df['mother_ethnic_origin']
        father_ethnic = df['father_ethnic_origin']
        combined = f"Mother: {mother_ethnic}. Father: {father_ethnic}"
        ethnic_fields.append(combined)
    if 'donor_race' in df.columns:
        ethnic_fields.append(df['donor_race'])
    if 'race' in df.columns:
        ethnic_fields.append(df['race'])
    if 'ancestry' in df.columns:
        ethnic_fields.append(df['ancestry'])
    if 'ethnic background' in df.columns:
        ethnic_fields.append(df['ethnic background'])
    
    standardized['ethnic_background'] = ethnic_fields[0] if ethnic_fields else np.nan
    
    # Handle religion
    if 'religion' in df.columns:
        standardized['religion'] = df['religion']
    else:
        standardized['religion'] = np.nan
    
    # Handle occupation
    if 'occupation' in df.columns:
        standardized['occupation'] = df['occupation']
    else:
        standardized['occupation'] = np.nan
    
    # Handle interests/hobbies
    if 'interests_and_hobbies' in df.columns:
        standardized['interests_hobbies'] = df['interests_and_hobbies']
    elif 'talents/hobbies' in df.columns:
        standardized['interests_hobbies'] = df['talents/hobbies']
    elif 'interests' in df.columns:
        standardized['interests_hobbies'] = df['interests']
    else:
        standardized['interests_hobbies'] = np.nan
    
    return standardized

def main():
    # Define file paths
    file_paths = {
        'bank1': '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/bank1_processed_data.csv',
        'bank2': '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/bank2_processed_data.csv',
        'bank3': '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/bank3_processed_data.csv',
        'bank4': '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/bank4_processed_data.csv'
    }
    
    # Read all datasets
    dfs = read_datasets(file_paths)
    
    # Clean and standardize each dataset
    standardized_dfs = []
    for bank, df in dfs.items():
        print(f"\nProcessing {bank}...")
        standardized_df = clean_and_standardize_df(df, bank)
        standardized_dfs.append(standardized_df)
        print(f"Completed processing {bank}")
        
    # Concatenate all standardized dataframes
    print("\nMerging all datasets...")
    final_df = pd.concat(standardized_dfs, ignore_index=True)
    
    # Save full version with IDs for validation
    validation_path = Path('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/merged_donor_data_with_ids.csv')
    final_df.to_csv(validation_path, index=False)
    
    # Save clean version without IDs and bank source
    clean_df = final_df.drop(['donor_id', 'bank_source'], axis=1)
    clean_path = Path('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/merged_donor_data.csv')
    clean_df.to_csv(clean_path, index=False)
    
    print(f"\nValidation dataset saved to: {validation_path}")
    print(f"Clean dataset saved to: {clean_path}")
    print(f"Total number of records: {len(final_df)}")
    print("\nColumn statistics:")
    print(final_df.info())
    print("\nRecords per bank:")
    print(final_df['bank_source'].value_counts())
    print("\nSample of cleaned descriptions:")
    print(final_df['donor_description'].head())
    print("\nMissing value counts:")
    print(final_df.isna().sum())

if __name__ == "__main__":
    main()


Columns in bank1:
['headline', 'donor_description', 'donor_lookalikes', 'height', 'weight', 'eye_color', 'hair_color', 'hair_texture', 'ethnic_origin', 'religion', 'jewish_ancestry', 'education_level', 'areas_of_study']

Columns in bank2:
['donor_description', 'donor_quote', 'interests_and_hobbies', 'skills', 'mother_ethnic_origin', 'father_ethnic_origin', 'donor_race', 'skin_tone', 'eye_color', 'hair_color', 'height', 'weight', 'education', 'occupation', 'nationality', 'religion']

Columns in bank3:
['donor_alias', 'height', 'weight', 'race', 'staff_impression']

Columns in bank4:
['description', 'height', 'weight', 'eye color', 'skin tone', 'hair', 'ancestry', 'ethnic background', 'degree', 'occupation', 'interests', 'age range at donation', 'astrological sign', 'favorite subject', 'religion', 'favorite pet', 'personal goals', 'talents/hobbies']

Processing bank1...
Completed processing bank1

Processing bank2...
Completed processing bank2

Processing bank3...
Completed processing b

## Convert .csv File into .json

In [11]:
import pandas as pd
from pathlib import Path
import json

def convert_csv_to_json():
    # Read the CSV file
    input_path = Path('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/merged_donor_data.csv')
    output_path = Path('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/merged_donor_data.json')
    
    # Read CSV
    df = pd.read_csv(input_path)
    
    # Convert DataFrame to list of dictionaries
    # orient='records' creates an array of objects where each object is a row
    json_data = json.loads(df.to_json(orient='records'))
    
    # Write to JSON file with proper formatting
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    
    # Print some statistics
    print(f"Converted {len(json_data)} records to JSON")
    print(f"Output saved to: {output_path}")
    
    # Show sample of first record
    print("\nSample record structure:")
    print(json.dumps(json_data[0], indent=2))

if __name__ == "__main__":
    convert_csv_to_json()

Converted 1019 records to JSON
Output saved to: /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/processed/merged_donor_data.json

Sample record structure:
{
  "donor_description": "Doctor & Baker. always goes out of his way to do the right thing and help others, especially with their health. You might say that\u2019s why he\u2019s so dedicated to earning his degree as a Doctor of Pharmacy (3.99 GPA). Outside of his professional life, he enjoys many fascinating hobbies: learning to play the harmonium, gardening, and classic literature. With wavy, dark hair, this handsome donor also shows he cares by cooking and baking for others \u2014 skills he learned from his mother and grandmother.",
  "height": "5'9 (175cm)",
  "weight": "173 lbs (78kg)",
  "eye_color": "Brown",
  "hair_color": "Dark Brown",
  "skin_tone": null,
  "education_level": "Postgraduate",
  "education_field": "Pharmacy",
  "ethnic_background": "East Indian",
  "religion": "Hindu",
  "occupation": null